In [ ]:
%pip install --upgrade -r requirements.txt

Create the Pay-i and Anthropic clients

In [ ]:
from anthropic import Anthropic 

from payi import Payi
from payi.lib.instrument import payi_instrument, ingest

import os

#Read the API KEYs from the environment, replace the default values (the second argument) with your own keys if needed
anthropic_key = os.getenv("ANTHROPIC_API_KEY", "YOUR_ANTHROPIC_API_KEY")
payi_key = os.getenv("PAYI_API_KEY", "YOUR_PAYI_API_KEY")

payi = Payi(
    api_key=payi_key,
)
 
# Instrument the anthropic client so that the @ingest decorator can be used
payi_instrument(payi)

client = Anthropic(
    api_key=anthropic_key,
)

Create an experience type and limit.  Both will be used when ingesting data into Pay-i

In [ ]:
experience_name = 'anthropic_ingest_experience_test'
payi.experiences.types.create(name=experience_name, description='This is a test experience')
 
result = payi.limits.create(max=10_000, limit_name='anthropic_ingest_limit')
limit_id = result.limit.limit_id

print(f"Created limit with id {limit_id}")

Decorate a method which calls the Anthropic client

In [4]:
from payi.lib.helpers import create_headers

# The typical decorator usage pattern is to specify request tags and/or experience name as they the same values will be applied to all Anthropic calls in the decorated function,
# while user_id and limits are typically specified in each Anthropic call to attribute usage to a user and/or limit

# When experience_name is provided and experience_id is not, the experience instance id will be automatically created and the same ID will be applied to all Anthropic calls
@ingest(request_tags=['x', 'y'],  experience_name=experience_name)
def call_anthropic(client, limit_id, user_id):
    
    message = client.messages.create(
        model="claude-3-5-sonnet-20241022",
        max_tokens=1000,
        temperature=0,
        system="You are a world-class poet. Respond only with short poems.",
        messages=[
            {
                "role": "user",
                "content": [
                    {
                        "type": "text",
                        "text": "Why is the ocean salty?"
                    }
                ]
            }
        ],
        # To pass per method metadata to ingest, specify each metadata value in the call to create_headers.  
        # These additional header values will not be sent to Anthropic.
        extra_headers=create_headers(user_id=user_id, limit_ids=[limit_id])
    )
    print(message.content)

    # Additional calls to anthropic can be made here and will be ingested.
    # If another method is called, it's calls to Anthropic will also be ingested.
    ...


Call the decorated method to call Anthropic and ingest the results through the Pay-i instrumentation

In [ ]:
call_anthropic(client, limit_id, 'a_user_id')

See that the limit input, output, and total base cost has been updated with the ingest data

In [ ]:
response = payi.limits.retrieve(limit_id=limit_id)
print(f"Limit Name: {response.limit.limit_name}")
print(f"Limit ID: {response.limit.limit_id}")
print(f"Limit Creation Timestamp: {response.limit.limit_creation_timestamp}")
print(f"Limit Tags: {response.limit.limit_tags}")
print(f"Limit Input Base Cost: {response.limit.totals.cost.input.base}")
print(f"Limit Output Base Cost: {response.limit.totals.cost.output.base}")
print(f"Limit Total Base Cost: {response.limit.totals.cost.output.base}")

Reset a limit back to zero tracked cost

In [ ]:
print("\nState after reset:")
response = payi.limits.retrieve(limit_id=limit_id)
print(f"Limit Name: {response.limit.limit_name}")
print(f"Limit ID: {response.limit.limit_id}")
print(f"Limit Creation Timestamp: {response.limit.limit_creation_timestamp}")
print(f"Limit Tags: {response.limit.limit_tags}")
print(f"Limit Input Base Cost: {response.limit.totals.cost}")
print(f"Limit Output Base Cost: {response.limit.totals.cost.output.base}")
print(f"Limit Total Base Cost: {response.limit.totals.cost.total.base}")